In [2]:
import pandas as pd
from pulp import *

#### Nutrition Facts

In [27]:
# Import Costs
nutrition = pd.read_excel('Nutrition Facts.xlsx', index_col = 0)
nutrition

,Protein,Fat,Fibre,Salt,Sugar
Ingredients,,,,,
Chicken,0.100000,0.080000,0.001000,0.002,0.000
Beef,0.200000,0.100000,0.005000,0.005,0.000
Mutton,0.150000,0.110000,0.003000,0.007,0.000
Rice,0.000000,0.010000,0.100000,0.002,0.000
Wheat bran,0.040000,0.010000,0.150000,0.008,0.000
Corn,0.032927,0.012805,0.028049,0.000,0.045
Peanuts,0.258000,0.492000,0.085000,0.001,0.047


#### Ingredients Costs

In [29]:
# Import Costs
costs = pd.read_excel('Costs.xlsx')
dict_costs = dict(zip(costs['Ingredients'], costs['Costs']))
costs

,Ingredients,Costs
0,Chicken,0.095
1,Beef,0.150
2,Mutton,0.100
3,Rice,0.002
4,Wheat bran,0.005
5,Corn,0.012
6,Peanuts,0.013


#### Build your model

In [53]:
# Variables
variables = ['Chicken', 'Beef', 'Mutton', 'Rice', 'Wheat bran', 'Corn', 'Peanuts']

# Initialize Class
model = LpProblem("Optimize your Protein Bar", LpMinimize)

# Create Decision Variables
x = LpVariable.dicts("Qty", [j for j in variables],
                     lowBound=0, upBound=None, cat='continuous')

# Define Objective Function
model += (lpSum([dict_costs[i] * x[i] for i in variables]))


# Add Constraints
model += (lpSum([x[i] for i in variables])) == 100
model += (lpSum([x[i] * nutrition.loc[i, 'Protein'] for i in variables])) >= 22
model += (lpSum([x[i] * nutrition.loc[i, 'Fat'] for i in variables])) <= 22
model += (lpSum([x[i] * nutrition.loc[i, 'Fibre'] for i in variables])) >= 6
model += (lpSum([x[i] * nutrition.loc[i, 'Salt'] for i in variables])) <= 3
model += (lpSum([x[i] * nutrition.loc[i, 'Sugar'] for i in variables])) <= 20
                                                   
                                                       
# Solve Model
model.solve()
print("Cost per Bar = {:,} $".format(round(value(model.objective), 2)))
print('\n' + "Status: {}".format(LpStatus[model.status]))
for v in model.variables():
    print(v.name, "=", round(v.varValue,2), 'g')

Cost per Bar = 11.08 $

Status: Infeasible
Qty_Beef = 71.27 g
Qty_Chicken = 0.0 g
Qty_Corn = 0.0 g
Qty_Mutton = 0.0 g
Qty_Peanuts = 30.26 g
Qty_Rice = 0.0 g
Qty_Wheat_bran = -1.53 g


C:\Users\Samir\Anaconda3\lib\site-packages\pulp\pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
